In [3]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4

cpu


In [1]:
with open("wizard_of_oz.txt", "r", encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [18]:
string_to_int  = { ch:i for i, ch in enumerate(chars)}
int_to_string   = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([84, 30, 65, 62, 61, 62, 65, 68, 55,  8,  1, 62, 55, 57, 55, 64, 54, 69,
         8,  1, 63, 75, 70, 58, 69,  1, 51, 64, 54,  1, 56, 51, 59, 68, 75,  1,
        70, 51, 62, 55, 69,  1, 58, 51, 72, 55,  1, 56, 65, 62, 62, 65, 73, 55,
        54,  1, 53, 58, 59, 62, 54, 58, 65, 65, 54,  0, 70, 58, 68, 65, 71, 57,
        58,  1, 70, 58, 55,  1, 51, 57, 55, 69,  8,  1, 56, 65, 68,  1, 55, 72,
        55, 68, 75,  1, 58, 55, 51, 62, 70, 58])


In [19]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [20]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([84]) target is tensor(30)
when input is tensor([84, 30]) target is tensor(65)
when input is tensor([84, 30, 65]) target is tensor(62)
when input is tensor([84, 30, 65, 62]) target is tensor(61)
when input is tensor([84, 30, 65, 62, 61]) target is tensor(62)
when input is tensor([84, 30, 65, 62, 61, 62]) target is tensor(65)
when input is tensor([84, 30, 65, 62, 61, 62, 65]) target is tensor(68)
when input is tensor([84, 30, 65, 62, 61, 62, 65, 68]) target is tensor(55)
